1.数据加载与切分

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# --- 我们先重新加载和切分数据，确保所有步骤都在一个代码块里 ---
# ## 1. 数据加载与切分

# 加载数据
file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
train_full = pd.read_csv(file_path)

# 分离特征 (X) 和目标 (y)
X_full = train_full.drop('SalePrice', axis=1)
y_full = train_full['SalePrice']

# --- **就放在这里！** ---
# 在切分之前，就从总的特征集里把Id去掉
X_full_no_id = X_full.drop('Id', axis=1)
# -------------------------

# 然后，用不含Id的特征集去进行切分
X_train, X_val, y_train, y_val = train_test_split(X_full_no_id, y_full, test_size=0.2, random_state=42)

2.对数变换

In [2]:

# 对目标变量y进行对数变换
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

3.特征工程

3.1 初级特征工程

In [3]:
import numpy as np
import pandas as pd

# 这是一个独立的函数，我们可以反复调用它
def feature_engineer_advanced(df):
    """
    对数据集进行高级特征工程，包括智能填充和特征创造。
    """
    # 为了不修改原始数据，先创建副本
    df_fe = df.copy()

    # --- 1. 智能填充：NaN 代表 "没有" ---
    
    # 填充类别型特征
    for col in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                'MasVnrType']:
        df_fe[col] = df_fe[col].fillna('None')

    # 填充数值型特征
    for col in ['GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2',
                'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                'MasVnrArea']:
        df_fe[col] = df_fe[col].fillna(0)

    # --- 2. 智能填充：用分组中位数填充 "真缺失" ---

    # LotFrontage 很可能和街区(Neighborhood)有关
    # 我们用每个街区LotFrontage的中位数来填充该街区的缺失值
    df_fe['LotFrontage'] = df_fe.groupby('Neighborhood')['LotFrontage'].transform(
        lambda x: x.fillna(x.median())
    )

    # 对于其他一些"真缺失"，我们暂时还用众数填充，作为保底策略
    for col in ['MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']:
        df_fe[col] = df_fe[col].fillna(df_fe[col].mode()[0])


    # --- 3. 创造新特征 (和我们上次做的一样) ---

    df_fe['TotalSF'] = df_fe['TotalBsmtSF'] + df_fe['1stFlrSF'] + df_fe['2ndFlrSF']
    df_fe['HouseAge'] = df_fe['YrSold'] - df_fe['YearBuilt']
    df_fe['RemodAge'] = df_fe['YrSold'] - df_fe['YearRemodAdd']
    df_fe['TotalBath'] = df_fe['BsmtFullBath'] + (0.5 * df_fe['BsmtHalfBath']) + \
                         df_fe['FullBath'] + (0.5 * df_fe['HalfBath'])
    
    # --- 4. (可选) 删除一些不再需要的原始列 ---
    # 比如我们已经有了HouseAge，YearBuilt可能就不那么重要了
    # df_fe = df_fe.drop(['YearBuilt', 'YearRemodAdd'], axis=1)

    print("✅ 高级特征工程函数已定义。")
    return df_fe

# --- 现在，我们用这个新函数来处理我们的训练集和验证集 ---
# 假设 X_train 和 X_val 已经存在
X_train_advanced = feature_engineer_advanced(X_train)
X_val_advanced = feature_engineer_advanced(X_val)

print("\n查看一下处理后的数据，确认缺失值是否已被处理：")
# 检查处理后的训练集是否还有缺失值
missing_after = X_train_advanced.isnull().sum().sum()
print(f"处理后，训练集剩余缺失值总数: {missing_after}")

✅ 高级特征工程函数已定义。
✅ 高级特征工程函数已定义。

查看一下处理后的数据，确认缺失值是否已被处理：
处理后，训练集剩余缺失值总数: 0


In [13]:
import numpy as np
import pandas as pd
from scipy.stats import skew

def feature_engineer_final_robust(df, train_df_for_skew_calc=None, skew_list_to_apply=None):
    """
    对数据集进行最终的、更健壮的特征工程。
    遵循“先清洗、后计算”的原则。
    """
    df_fe = df.copy()

    # --- 阶段一：彻底清洗缺失值 ---

    # 1. 填充 "NaN" 代表 "没有" 的类别特征
    for col in ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType']:
        df_fe[col] = df_fe[col].fillna('None')

    # 2. 填充 "NaN" 代表 "0" 的数值特征
    for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea']:
        df_fe[col] = df_fe[col].fillna(0)

    # 3. 填充 "真缺失" 的类别特征 (用众数)
    for col in ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'Electrical', 'KitchenQual', 'Functional', 'SaleType']:
        if col in df_fe.columns:
            df_fe[col] = df_fe[col].fillna(df_fe[col].mode()[0])

    # 4. 填充 "真缺失" 的数值特征 (用中位数)
    # 这是一个“保底”步骤，确保所有我们没明确处理的数值缺失值都被填充
    numerical_cols_with_na = df_fe.select_dtypes(include=np.number).columns[df_fe.select_dtypes(include=np.number).isnull().any()]
    for col in numerical_cols_with_na:
        df_fe[col] = df_fe[col].fillna(df_fe[col].median())


    # --- 阶段二：在干净的数据上进行特征创造与变换 ---

    # 5. 基础特征创造
    df_fe['TotalSF'] = df_fe['TotalBsmtSF'] + df_fe['1stFlrSF'] + df_fe['2ndFlrSF']
    df_fe['HouseAge'] = df_fe['YrSold'] - df_fe['YearBuilt']
    df_fe['RemodAge'] = df_fe['YrSold'] - df_fe['YearRemodAdd']
    df_fe['TotalBath'] = df_fe['BsmtFullBath'] + (0.5 * df_fe['BsmtHalfBath']) + df_fe['FullBath'] + (0.5 * df_fe['HalfBath'])
    
    # 6. 处理年龄为负数的边界情况
    df_fe['HouseAge'] = df_fe['HouseAge'].clip(0)
    df_fe['RemodAge'] = df_fe['RemodAge'].clip(0)

    # 7. 创建交互特征
    df_fe['Qual_x_TotalSF'] = df_fe['OverallQual'] * df_fe['TotalSF']
    df_fe['Qual_x_HouseAge'] = df_fe['OverallQual'] * df_fe['HouseAge']
    
    # 8. 处理数值特征倾斜度 (正态化)
    if train_df_for_skew_calc is not None:
        numerical_feats = train_df_for_skew_calc.select_dtypes(exclude="object").columns
        skewed_feats = train_df_for_skew_calc[numerical_feats].apply(lambda x: skew(x.dropna()))
        skew_list_to_apply = skewed_feats[skewed_feats > 0.75].index
        print(f"从训练集中发现 {len(skew_list_to_apply)} 个高度倾斜的特征将被变换。")
    
    if skew_list_to_apply is not None:
        for feat in skew_list_to_apply:
            if feat in df_fe.columns:
                df_fe[feat] = np.log1p(df_fe[feat])
    
    return df_fe, skew_list_to_apply

# --- 现在，我们用这个最终修正版的函数来处理数据 ---
# 假设 X_train 和 X_val 已经存在

# 1. 在训练集上应用，并获取倾斜特征列表
X_train_final, skew_list = feature_engineer_final_robust(X_train, train_df_for_skew_calc=X_train)

# 2. 在验证集上应用，并传入从训练集学到的倾斜列表
X_val_final, _ = feature_engineer_final_robust(X_val, skew_list_to_apply=skew_list)


print("\n✅ 终极健壮版特征工程执行完毕！")
missing_after = X_train_final.isnull().sum().sum()
print(f"处理后，训练集剩余缺失值总数: {missing_after}")

从训练集中发现 21 个高度倾斜的特征将被变换。

✅ 终极健壮版特征工程执行完毕！
处理后，训练集剩余缺失值总数: 0


3.2 暂时最终版特征工程

3.2.1 清洗

In [14]:
# ## 1. 数据加载、清洗与切分

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# 加载数据
file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
train_full = pd.read_csv(file_path)

# 【新增】移除著名的异常值
outlier_indices = train_full[(train_full['GrLivArea'] > 4000) & (train_full['SalePrice'] < 300000)].index
train_full = train_full.drop(outlier_indices)
print(f"移除了 {len(outlier_indices)} 个GrLivArea异常值。")

# 分离特征和目标
X_full = train_full.drop('SalePrice', axis=1)
y_full = train_full['SalePrice']

# 舍去Id列
X_full = X_full.drop('Id', axis=1)

# 切分数据
X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

# 对目标变量y进行对数变换
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

print("数据加载、异常值移除、切分完成！")

移除了 2 个GrLivArea异常值。
数据加载、异常值移除、切分完成！


3.2.2 特征工程函数

In [16]:
import numpy as np
import pandas as pd
from scipy.stats import skew

def feature_engineer_ultimate_final(df, skew_list_to_apply=None):
    """
    终极修正版特征工程函数，修复了类型转换和倾斜度计算的顺序问题。
    """
    df_fe = df.copy()

    # --- 1. 类型转换 (必须在最前面) ---
    df_fe['MSSubClass'] = df_fe['MSSubClass'].astype(str)

    # --- 2. 有序特征的数值映射 ---
    quality_map = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
    ordered_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
    for col in ordered_cols:
        if col in df_fe.columns:
            df_fe[col] = df_fe[col].map(quality_map)

    # --- 3. 智能填充 ---
    # (这部分代码保持不变)
    for col in ['Alley', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Fence', 'GarageType', 'GarageFinish', 'MasVnrType', 'MiscFeature']:
         df_fe[col] = df_fe[col].fillna('None')
    for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea'] + ordered_cols:
        df_fe[col] = df_fe[col].fillna(0)
    df_fe['LotFrontage'] = df_fe.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    for col in ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'Electrical', 'Functional', 'SaleType']:
        if col in df_fe.columns: df_fe[col] = df_fe[col].fillna(df_fe[col].mode()[0])
    
    # 对任何可能遗漏的数值列进行中位数填充
    numerical_cols_with_na = df_fe.select_dtypes(include=np.number).columns[df_fe.select_dtypes(include=np.number).isnull().any()]
    for col in numerical_cols_with_na:
        df_fe[col] = df_fe[col].fillna(df_fe[col].median())


    # --- 4. 基础特征创造 ---
    # (这部分代码保持不变)
    df_fe['TotalSF'] = df_fe['TotalBsmtSF'] + df_fe['1stFlrSF'] + df_fe['2ndFlrSF']
    df_fe['HouseAge'] = df_fe['YrSold'] - df_fe['YearBuilt']
    df_fe['RemodAge'] = df_fe['YrSold'] - df_fe['YearRemodAdd']
    df_fe['TotalBath'] = df_fe['BsmtFullBath'] + (0.5 * df_fe['BsmtHalfBath']) + df_fe['FullBath'] + (0.5 * df_fe['HalfBath'])

    # --- 5. 处理年龄为负数的边界情况 ---
    # (这部分代码保持不变)
    df_fe['HouseAge'] = df_fe['HouseAge'].clip(0)
    df_fe['RemodAge'] = df_fe['RemodAge'].clip(0)

    # --- 6. 创建交互特征 ---
    # (这部分代码保持不变)
    df_fe['Qual_x_TotalSF'] = df_fe['OverallQual'] * df_fe['TotalSF']
    df_fe['Qual_x_HouseAge'] = df_fe['OverallQual'] * df_fe['HouseAge']
    
    # --- 7. 处理数值特征倾斜度 (正态化) ---
    # 【修正】: 在转换后的数据上识别数值列
    if skew_list_to_apply is None: # 如果是训练集...
        print("正在计算训练集的数值特征倾斜度...")
        # 核心修正：在已经转换了MSSubClass的df_fe上识别数值列
        numerical_feats = df_fe.select_dtypes(exclude=["object", "category"]).columns
        skewed_feats = df_fe[numerical_feats].apply(lambda x: skew(x.dropna()))
        skew_list_to_apply = skewed_feats[skewed_feats > 0.5].index
        print(f"发现 {len(skew_list_to_apply)} 个高度倾斜的特征。")
    
    # 对所有倾斜的特征应用log1p变换
    for feat in skew_list_to_apply:
        df_fe[feat] = np.log1p(df_fe[feat])
    
    return df_fe, skew_list_to_apply

# --- 调用最终版函数 (调用方式不变) ---
X_train_pro, skew_list_pro = feature_engineer_ultimate_final(X_train)
X_val_pro, _ = feature_engineer_ultimate_final(X_val, skew_list_to_apply=skew_list_pro)

print("\n✅ 终极Pro版特征工程执行完毕！")

正在计算训练集的数值特征倾斜度...
发现 31 个高度倾斜的特征。

✅ 终极Pro版特征工程执行完毕！


4.预处理pipeline

In [4]:
# --- ## 4. 数据预处理流水线 (Preprocessing Pipeline) ---

# 首先，我们从经过特征工程的数据中舍去Id列
# （虽然我们之前的函数没用到Id，但在这里再确认一遍是好习惯）
if 'Id' in X_train_advanced.columns:
    X_train_final = X_train_advanced.drop('Id', axis=1)
    X_val_final = X_val_advanced.drop('Id', axis=1)
else:
    X_train_final = X_train_advanced
    X_val_final = X_val_advanced

# 1. 重新定义数值列和类别列的列表
# 因为我们增加了新特征（如TotalSF），所以需要更新列表
categorical_cols = [cname for cname in X_train_final.columns if X_train_final[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_final.columns if X_train_final[cname].dtype in ['int64', 'float64']]

# 2. 创建预处理管道 (和之前一样，但现在作用于新数据)
# 对于数值型特征，我们只保留一个中位数填充器，以防万一有未处理的缺失值
numerical_transformer = SimpleImputer(strategy='median')

# 对于类别型特征，我们只需要进行独热编码
# 因为我们的特征工程函数已经填充了所有类别缺失值
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. 组合成一个总的预处理器
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# 4. 用预处理器来处理经过高级特征工程的数据
X_train_processed = preprocessor.fit_transform(X_train_final)
X_val_processed = preprocessor.transform(X_val_final)


# 5. 查看最终处理后的数据形状
print("高级特征工程后，训练集形状:", X_train_final.shape)
print("最终处理后，训练集形状:", X_train_processed.shape)
print("\n高级特征工程后，验证集形状:", X_val_final.shape)
print("最终处理后，验证集形状:", X_val_processed.shape)

print("\n✅ 所有数据准备工作已完成！现在可以进入模型训练了。")

高级特征工程后，训练集形状: (1168, 83)
最终处理后，训练集形状: (1168, 304)

高级特征工程后，验证集形状: (292, 83)
最终处理后，验证集形状: (292, 304)

✅ 所有数据准备工作已完成！现在可以进入模型训练了。


4# 预处理最终版

In [37]:
# ## 4. 数据预处理流水线 (最终健壮版)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# 我们的输入是 X_train_pro 和 X_val_pro

# 1. 重新定义数值列和类别列的列表 (不变)
categorical_cols = [cname for cname in X_train_pro.columns if X_train_pro[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train_pro.columns if X_train_pro[cname].dtype in ['int64', 'float64']]

# --- 【核心修正】 ---
# 2. 从训练集中为每个类别特征提取“标准答案”（所有可能的类别）
categories_list = [X_train_pro[col].unique().tolist() for col in categorical_cols]

# 3. 创建预处理管道
# 数值管道不变
numerical_transformer = SimpleImputer(strategy='median')

# 类别管道现在使用我们提供的“标准答案”
categorical_transformer = Pipeline(steps=[
    # 明确告知编码器所有可能的类别
    ('onehot', OneHotEncoder(handle_unknown='ignore', categories=categories_list))
])
# --------------------

# 4. 组合成一个总的预处理器
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough'
)

# 5. 用预处理器来处理我们的“终极”特征数据
X_train_processed_final = preprocessor.fit_transform(X_train_pro)
X_val_processed_final = preprocessor.transform(X_val_pro)

# 6. 检查最终处理后的数据形状，确保它们一致
print("终极特征工程后，训练集形状:", X_train_pro.shape)
print("最终预处理后，训练集形状:", X_train_processed_final.shape)
print("\n终极特征工程后，验证集形状:", X_val_pro.shape)
print("最终预处理后，验证集形状:", X_val_processed_final.shape)

# 检查列数是否一致
if X_train_processed_final.shape[1] == X_val_processed_final.shape[1]:
    print("\n✅ 特征数量一致！问题已解决。")
else:
    print("\n‼️ 警告：特征数量仍然不一致，请检查代码。")

终极特征工程后，训练集形状: (1166, 85)
最终预处理后，训练集形状: (1166, 278)

终极特征工程后，验证集形状: (292, 85)
最终预处理后，验证集形状: (292, 278)

✅ 特征数量一致！问题已解决。


5.1.a 使用random forest

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np

# --- 再次训练模型，但这次用的是经过高级特征工程的数据 ---

# 我们可以叫它 model_v2，以区别于之前的基准模型
model_v2 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

print("正在使用加强版特征数据训练新模型...")
# ‼️注意：我们用的是 X_train_processed 和 y_train_log
model_v2.fit(X_train_processed, y_train_log)
print("✅ 新模型训练完成！")

# 在验证集上进行预测
val_preds_log_v2 = model_v2.predict(X_val_processed)

# 将预测值逆变换回原始尺度
val_preds_v2 = np.expm1(val_preds_log_v2)

# 计算并打印新的评估分数
rmsle_v2 = np.sqrt(mean_squared_log_error(y_val, val_preds_v2))

print("-" * 50)
print(f"我们之前的基准分数为: 0.14561")
print(f"🎉 使用高级特征工程后，新分数为: {rmsle_v2:.5f}")
print("-" * 50)

正在使用加强版特征数据训练新模型...
✅ 新模型训练完成！
--------------------------------------------------
我们之前的基准分数为: 0.14561
🎉 使用高级特征工程后，新分数为: 0.14322
--------------------------------------------------


5.1.b.1 使用lightGBM

In [39]:
# LightGBM 是一个需要单独安装的库
# 在Kaggle环境中，通常已经预装好了
# 如果在本地，你可能需要运行 !pip install lightgbm
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
import numpy as np

# --- 使用相同的“加强版”数据，但更换模型 ---

# 1. 定义 LightGBM 回归器模型
# 同样设置 random_state 以保证结果可复现
model_lgbm = lgb.LGBMRegressor(random_state=42)

print("正在使用加强版特征数据训练 LightGBM 模型...")
# ‼️注意：我们用的仍然是 X_train_processed 和 y_train_log
model_lgbm.fit(X_train_processed, y_train_log)
print("✅ LightGBM 模型训练完成！")

# 2. 在验证集上进行预测
val_preds_log_lgbm = model_lgbm.predict(X_val_processed)

# 3. 将预测值逆变换回原始尺度
val_preds_lgbm = np.expm1(val_preds_log_lgbm)

# 4. 计算并打印新的评估分数
rmsle_lgbm = np.sqrt(mean_squared_log_error(y_val, val_preds_lgbm))

print("-" * 50)
print(f"随机森林模型的分数是: 0.14544")
print(f"🚀 使用 LightGBM 模型后，新分数为: {rmsle_lgbm:.5f}")
print("-" * 50)

正在使用加强版特征数据训练 LightGBM 模型...
✅ LightGBM 模型训练完成！
--------------------------------------------------
随机森林模型的分数是: 0.14544
🚀 使用 LightGBM 模型后，新分数为: 0.13299
--------------------------------------------------


5.1.b.2 lightGBM超参数调优

In [40]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_log_error

# --- 使用我们之前处理好的、最强的数据 ---
# 确保 X_train_processed, y_train_log, X_val_processed, y_val 都已存在

print("开始进行LightGBM的超参数调优...")

# 1. 定义我们要搜索的参数网格
# 我们只选几个关键参数进行一个初步的、较快的搜索
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.05, 0.1],
    'num_leaves': [20, 31, 40],
    'random_state': [42] # 固定随机种子以复现结果
}

# 2. 初始化LGBM模型和GridSearchCV
lgbm = lgb.LGBMRegressor(verbosity=-1)
grid_search = GridSearchCV(estimator=lgbm, 
                           param_grid=param_grid, 
                           cv=3, # 3折交叉验证以节省时间
                           scoring='neg_root_mean_squared_error', # 使用负的RMSE作为评分标准
                           n_jobs=-1, # 使用所有CPU核心
                           verbose=1) # 打印搜索过程

# 3. 在【对数变换后】的训练数据上进行搜索
# GridSearchCV会自动处理交叉验证，所以我们用全部的训练数据
grid_search.fit(X_train_processed, y_train_log)

print("\n✅ 超参数调优完成！")
print("-" * 50)
print(f"找到的最佳参数组合是: {grid_search.best_params_}")
print(f"在交叉验证中得到的最佳分数为: {-grid_search.best_score_:.5f}") # 分数要取反
print("-" * 50)


# 4. 使用找到的最佳模型在验证集上进行最终评估
best_lgbm_model = grid_search.best_estimator_
val_preds_log_best = best_lgbm_model.predict(X_val_processed)
val_preds_best_lgbm= np.expm1(val_preds_log_best)

rmsle_best = np.sqrt(mean_squared_log_error(y_val, val_preds_best_lgbm))

print(f"我们之前的单模型分数是: 0.13702")
print(f"⚙️ 经过精细调优后，新分数为: {rmsle_best:.5f}")
print("-" * 50)

开始进行LightGBM的超参数调优...
Fitting 3 folds for each of 18 candidates, totalling 54 fits

✅ 超参数调优完成！
--------------------------------------------------
找到的最佳参数组合是: {'learning_rate': 0.05, 'n_estimators': 200, 'num_leaves': 20, 'random_state': 42}
在交叉验证中得到的最佳分数为: 0.12836
--------------------------------------------------
我们之前的单模型分数是: 0.13702
⚙️ 经过精细调优后，新分数为: 0.12969
--------------------------------------------------


5.1.c.1 使用XGBoost

In [41]:
# XGBoost 也是一个需要单独安装的库
# 在Kaggle环境中，通常已经预装好了
# 如果在本地，你可能需要运行 !pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
import numpy as np

# --- 使用相同的“加强版”数据，但更换为XGBoost模型 ---

# 1. 定义 XGBoost 回归器模型
# 我们给它更多的树(n_estimators=1000)，但用更小的学习率(learning_rate=0.05)
# 同时设置了 early_stopping_rounds，让它在验证集上性能不再提升时自动停止训练
model_xgb = xgb.XGBRegressor(n_estimators=1000, 
                             learning_rate=0.05, 
                             random_state=42, 
                             n_jobs=-1,
                             early_stopping_rounds=5) # 核心：如果验证集分数连续5轮没有提升，就停止训练

print("正在使用加强版特征数据训练 XGBoost 模型...")

# 2. 训练模型，并加入提前停止的设置
# ‼️注意：为了使用early_stopping_rounds，我们需要在.fit()中提供验证集数据
model_xgb.fit(X_train_processed, y_train_log, 
              eval_set=[(X_val_processed, y_val_log)], 
              verbose=False) # verbose=False让它在训练中不打印过多信息

print("✅ XGBoost 模型训练完成！")

# 3. 在验证集上进行预测
val_preds_log_xgb = model_xgb.predict(X_val_processed)

# 4. 将预测值逆变换回原始尺度
val_preds_xgb = np.expm1(val_preds_log_xgb)

# 5. 计算并打印新的评估分数
# 这里需要处理一个边界情况：如果模型预测出负数（虽然罕见），会导致计算错误
# 我们确保所有预测值至少为0
val_preds_xgb[val_preds_xgb < 0] = 0
rmsle_xgb = np.sqrt(mean_squared_log_error(y_val, val_preds_xgb))

print("-" * 50)
print(f"LightGBM 模型的分数是: 0.13702")
print(f"🚀 使用 XGBoost 模型后，新分数为: {rmsle_xgb:.5f}")
print("-" * 50)

正在使用加强版特征数据训练 XGBoost 模型...
✅ XGBoost 模型训练完成！
--------------------------------------------------
LightGBM 模型的分数是: 0.13702
🚀 使用 XGBoost 模型后，新分数为: 0.13171
--------------------------------------------------


5.1.c.2 XGB调优

In [42]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_log_error

print("开始进行XGBoost的超参数调优...")

# 1. 为XGBoost定义参数网格
# 我们同样只选几个关键参数
xgb_param_grid = {
    'n_estimators': [200, 500],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'random_state': [42]
}

# 2. 初始化XGBoost模型和GridSearchCV
xgbr = xgb.XGBRegressor()
xgb_grid_search = GridSearchCV(estimator=xgbr, 
                               param_grid=xgb_param_grid, 
                               cv=3, 
                               scoring='neg_root_mean_squared_error',
                               n_jobs=-1,
                               verbose=1)

# 3. 在训练数据上进行搜索
xgb_grid_search.fit(X_train_processed, y_train_log)

print("\n✅ XGBoost超参数调优完成！")
print("-" * 50)
print(f"找到的最佳参数组合是: {xgb_grid_search.best_params_}")
print(f"在交叉验证中得到的最佳分数为: {-xgb_grid_search.best_score_:.5f}")
print("-" * 50)

# 4. 使用找到的最佳XGBoost模型在验证集上进行评估
best_xgb_model = xgb_grid_search.best_estimator_
val_preds_log_best_xgb = best_xgb_model.predict(X_val_processed)
val_preds_best_xgb = np.expm1(val_preds_log_best_xgb)
val_preds_best_xgb[val_preds_best_xgb < 0] = 0 # 保证非负

rmsle_best_xgb = np.sqrt(mean_squared_log_error(y_val, val_preds_best_xgb))

print(f"我们之前的XGBoost单模型分数是: 0.13893")
print(f"⚙️ 经过精细调优后，XGBoost的新分数为: {rmsle_best_xgb:.5f}")
print("-" * 50)

开始进行XGBoost的超参数调优...
Fitting 3 folds for each of 8 candidates, totalling 24 fits

✅ XGBoost超参数调优完成！
--------------------------------------------------
找到的最佳参数组合是: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 500, 'random_state': 42}
在交叉验证中得到的最佳分数为: 0.12107
--------------------------------------------------
我们之前的XGBoost单模型分数是: 0.13893
⚙️ 经过精细调优后，XGBoost的新分数为: 0.12242
--------------------------------------------------


5.1.d catboost调优

In [43]:
# CatBoost 也是一个需要单独安装的库
# 在Kaggle环境中，通常已经预装好了
# 如果在本地，你可能需要运行 !pip install catboost
import catboost as cb
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import mean_squared_log_error

# --- 我们使用特征工程后、独热编码前的数据 ---
# 确保 X_train_pro, y_train_log, X_val_pro, y_val 都已存在

print("开始进行CatBoost的超参数调优...")

# 1. 首先，我们需要明确告诉CatBoost哪些是类别特征
#    我们直接从 X_train_pro 数据框中获取类别列的列名
categorical_cols_names = [cname for cname in X_train_pro.columns if X_train_pro[cname].dtype == "object"]


# 2. 定义我们要搜索的参数网格 (不变)
cat_param_grid = {
    'iterations': [200, 500],
    'learning_rate': [0.05, 0.1],
    'depth': [4, 6],
    'random_state': [42]
}

# 3. 初始化CatBoost模型和GridSearchCV (不变)
model_cat = cb.CatBoostRegressor(cat_features=categorical_cols_names, verbose=0)
cat_grid_search = GridSearchCV(estimator=model_cat, 
                               param_grid=cat_param_grid, 
                               cv=3, 
                               scoring='neg_root_mean_squared_error',
                               n_jobs=-1)

# 4. 在【未经独热编码】的训练数据上进行搜索 (正确)
cat_grid_search.fit(X_train_pro, y_train_log)

print("\n✅ CatBoost超参数调优完成！")
print("-" * 50)
print(f"找到的最佳参数组合是: {cat_grid_search.best_params_}")
print(f"在交叉验证中得到的最佳分数为: {-cat_grid_search.best_score_:.5f}")
print("-" * 50)


# 5. 使用找到的最佳CatBoost模型在验证集上进行最终评估
best_cat_model = cat_grid_search.best_estimator_

# ‼️【修正】: 预测时也必须使用未经独热编码的 X_val_pro
val_preds_log_best_cat = best_cat_model.predict(X_val_pro)
val_preds_best_cat = np.expm1(val_preds_log_best_cat)

rmsle_best_cat = np.sqrt(mean_squared_log_error(y_val, val_preds_best_cat))

# 更新一下对比基准为我们当前最好的单模型分数
print(f"我们之前的最好成绩(调优后XGBoost)是: 0.12242")
print(f"🚀 使用调优后的 CatBoost 模型，新分数为: {rmsle_best_cat:.5f}")
print("-" * 50)

开始进行CatBoost的超参数调优...

✅ CatBoost超参数调优完成！
--------------------------------------------------
找到的最佳参数组合是: {'depth': 6, 'iterations': 500, 'learning_rate': 0.1, 'random_state': 42}
在交叉验证中得到的最佳分数为: 0.12230
--------------------------------------------------
我们之前的最好成绩(调优后XGBoost)是: 0.12242
🚀 使用调优后的 CatBoost 模型，新分数为: 0.12171
--------------------------------------------------


5.2 模型融合

In [44]:
# --- 假设以下变量来自你刚刚运行的、使用终极特征的几次模型训练 ---

# val_preds_best_cat (来自终极数据 + 调优后CatBoost, 分数 0.12171)
# val_preds_best_xgb (来自终极数据 + 调优后XGBoost, 分数 0.12242)
# val_preds_best_lgbm (来自终极数据 + 调优后LightGBM, 分数 0.12969)

# --- 让我们进行最终的加权融合 ---
# 我们给新科冠军CatBoost最高的权重
ultimate_blend_preds = 0.6 * val_preds_best_cat + 0.2 * val_preds_best_xgb + 0.2 * val_preds_best_lgbm
rmsle_ultimate_blend = np.sqrt(mean_squared_log_error(y_val, ultimate_blend_preds))

print("-" * 50)
print(f"调优后 CatBoost 单模型分数: 0.12171 (目前最好)")
print(f"调优后 XGBoost 单模型分数:  0.12242")
print(f"调优后 LightGBM 单模型分数: 0.12969")
print(f"👑👑👑 【终极融合】后，我们的毕业分数为: {rmsle_ultimate_blend:.5f}")
print("-" * 50)

--------------------------------------------------
调优后 CatBoost 单模型分数: 0.12171 (目前最好)
调优后 XGBoost 单模型分数:  0.12242
调优后 LightGBM 单模型分数: 0.12969
👑👑👑 【终极融合】后，我们的毕业分数为: 0.12005
--------------------------------------------------


6.最后生成预测结果

In [45]:
# 假设 preprocessor 和 model 变量已经存在于之前的代码块中

# 1. 加载官方测试数据
test_data_path = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
test_df = pd.read_csv(test_data_path)

# 2. 准备测试数据 (保存Id，并去除Id列)
test_ids = test_df['Id']
test_X = test_df.drop('Id', axis=1)

# 3. 使用【已经拟合过】的预处理器来转换测试数据
# ‼️注意：这里只用 .transform()，绝不能用 .fit_transform()
test_X_processed = preprocessor.transform(test_X)

# 4. 使用【已经训练好】的模型进行预测
test_preds_log = model.predict(test_X_processed)

# 5. 将预测结果逆变换回原始尺度
test_preds = np.expm1(test_preds_log)

# 6. 创建提交文件
submission_df = pd.DataFrame({'Id': test_ids, 'SalePrice': test_preds})

# 7. 保存为 .csv 文件
submission_df.to_csv('submission_baseline.csv', index=False)

print("-" * 50)
print("🎉 提交文件 'submission_baseline.csv' 已成功生成！")
print("现在你可以去Kaggle的 'Output' 部分找到它并提交了。")
print("-" * 50)

KeyError: "['TotalSF', 'HouseAge', 'RemodAge', 'TotalBath', 'Qual_x_TotalSF', 'Qual_x_HouseAge'] not in index"

In [46]:
# ## 6. 生成最终提交文件 (独立模块版)

import pandas as pd
import numpy as np

print("开始生成最终提交文件...")

# --- 1. 加载官方测试数据 ---
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test_ids = test_df['Id'] # 保存Id用于最终文件

# --- 2. 应用我们最强的特征工程函数 ---
# 核心：我们使用从训练集(X_train)中学到的skew_list_pro，应用到测试集上
X_test_final, _ = feature_engineer_ultimate_final(test_df, skew_list_to_apply=skew_list_pro)

# 从特征中去掉Id列，因为它不用于预测
if 'Id' in X_test_final.columns:
    X_test_final = X_test_final.drop('Id', axis=1)


# --- 3. 分别为不同模型准备数据并进行预测 ---

# a) CatBoost的预测 (在独热编码前的数据上)
print("CatBoost正在预测...")
preds_log_cat = best_cat_model.predict(X_test_final)
preds_cat = np.expm1(preds_log_cat)


# b) LGBM 和 XGBoost 的预测 (在独热编码后的数据上)
# 使用我们之前在训练集上拟合好的preprocessor来转换测试数据
print("预处理器正在转换测试数据...")
X_test_processed = preprocessor.transform(X_test_final)

print("LightGBM正在预测...")
preds_log_lgbm = best_lgbm_model.predict(X_test_processed)
preds_lgbm = np.expm1(preds_log_lgbm)

print("XGBoost正在预测...")
preds_log_xgb = best_xgb_model.predict(X_test_processed)
preds_xgb = np.expm1(preds_log_xgb)


# --- 4. 终极加权融合 ---
print("正在融合三个模型的预测结果...")
# 使用我们验证过的最佳权重
final_predictions = 0.6 * preds_cat + 0.2 * preds_xgb + 0.2 * preds_lgbm


# --- 5. 创建并保存提交文件 ---
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': final_predictions})
submission.to_csv('submission_final_blend.csv', index=False)

print("-" * 50)
print("🎉🎉🎉 最终提交文件 'submission_final_blend.csv' 已成功生成！🎉🎉🎉")
print("你可以去 'Output' 部分找到它并提交到Kaggle。")
print("-" * 50)

开始生成最终提交文件...
CatBoost正在预测...
预处理器正在转换测试数据...
LightGBM正在预测...
XGBoost正在预测...
正在融合三个模型的预测结果...
--------------------------------------------------
🎉🎉🎉 最终提交文件 'submission_final_blend.csv' 已成功生成！🎉🎉🎉
你可以去 'Output' 部分找到它并提交到Kaggle。
--------------------------------------------------


In [36]:
# 假设 X_train_pro 和 X_val_pro (或你最终命名的变量) 已经存在
# 我们用 X_train_pro 和 X_test_final (来自你上个代码块) 来比较

print("--- 开始排查训练集和测试集的类别差异 ---")

# 找出两个数据集中共有的类别列
common_categorical_cols = [col for col in categorical_cols if col in X_test_final.columns]

# 逐一比较每个类别列
for col in common_categorical_cols:
    train_categories = set(X_train_pro[col].unique())
    test_categories = set(X_test_final[col].unique())

    # 找出只在测试集中出现的新类别
    new_categories_in_test = test_categories - train_categories

    if new_categories_in_test:
        print(f"\n‼️ 在特征 '{col}' 中发现问题！")
        print(f"   -> 测试集出现了训练集中没有的新类别: {new_categories_in_test}")

print("\n--- 排查完毕 ---")

--- 开始排查训练集和测试集的类别差异 ---

‼️ 在特征 'MSSubClass' 中发现问题！
   -> 测试集出现了训练集中没有的新类别: {'150'}

‼️ 在特征 'Functional' 中发现问题！
   -> 测试集出现了训练集中没有的新类别: {'Sev'}

--- 排查完毕 ---
